# Edges vs triangles

In [ ]:
#@title Verification and visualization code

import numpy as np
from matplotlib import pyplot as plt


def sum_pairwise_triple_products_batch(
    A: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
  """Calculates the sum of pairwise and triple products for each row of a 2D array.

  Args:
    A: A 2D numpy array of shape (M, N), where M is the number of lists and N is
      the size of each list (e.g., 20).

  Returns:
    A tuple containing two 1D numpy arrays:
    - pairwise_sums: Array of shape (M,) with sum_{i<j} a_i*a_j for each row.
    - triple_sums: Array of shape (M,) with sum_{i<j<k} a_i*a_j*a_k for each
    row.
  """
  if A.ndim != 2:
    raise ValueError(
        f'Input array must be 2-dimensional, but got shape {A.shape}'
    )

  M, N = A.shape

  triple_sums = np.zeros(M, dtype=A.dtype)
  # Handle cases where products are not possible
  if N < 2:
    pairwise_sums = np.zeros(M, dtype=A.dtype)
    return pairwise_sums, triple_sums

  # Calculate power sums along axis 1 (rows)
  # S1 will have shape (M,)
  S1 = np.sum(A, axis=1)
  # np.square(A) is element-wise, result (M, N). Sum along axis 1 -> shape (M,)
  S2 = np.sum(np.square(A), axis=1)

  # Calculate pairwise sums (shape M,)
  # All operations (**, *) are element-wise on the (M,) arrays S1, S2
  pairwise_sums = np.square(S1) - S2

  # Calculate triple sums only if N >= 3
  if N >= 3:
    # np.power(A, 3) is element-wise, result (M, N). Sum along axis 1 -> shape (M,)
    S3 = np.sum(np.power(A, 3), axis=1)
    # All operations (**, *) are element-wise on the (M,) arrays S1, S2, S3
    triple_sums = np.power(S1, 3) - 3 * S1 * S2 + 2 * S3
  # else triple_sums was already initialized to zeros if N < 3

  return pairwise_sums, triple_sums


def analyze_density_curve(
        edge_densities: np.ndarray,
    triangle_densities: np.ndarray,
    gap_range_min: float = 0.0,
    gap_range_max: float = 1.0,
) -> tuple[float, float]:
    """Calculates area under piecewise f defined by slope 3 capped by y_{i+1},
    and finds the max gap.

    f(0)=0, f(1)=1. f(xi)=yi.
    For xi <= x < x_{i+1}:
    - If yi > y_{i+1}, f(x) = yi (horizontal).
    - If yi <= y_{i+1}:
        - Start with slope 3 from (xi, yi).
        - If slope 3 reaches y_{i+1} at or before x reaches x_{i+1}:
            f increases with slope 3 until y=y_{i+1}, then is horizontal at y_{i+1}
            until x=x_{i+1}.
        - If slope 3 does not reach y_{i+1} by x=x_{i+1}:
            f increases with slope 3 for the entire interval.

    Args:
        edge_densities: 1D array of edge densities (x-coordinates).
        triangle_densities: 1D array of triangle densities (y-coordinates).
        gap_range_min: Min edge density for gap analysis.
        gap_range_max: Max edge density for gap analysis.

    Returns:
        Tuple (area, max_gap_in_range). Area under the capped slope-3 func.
    """
    if (
        edge_densities.shape != triangle_densities.shape
        or edge_densities.ndim != 1
    ):
        print(f"Invalid input shapes: {edge_densities.shape}, {triangle_densities.shape}")
        return -1.0, -1.0 # Indicate error

    # Prepare points including boundaries (0,0) and (1,1)
    if edge_densities.size > 0:
        sort_indices = np.argsort(edge_densities)
        sorted_x = edge_densities[sort_indices]
        sorted_y = triangle_densities[sort_indices]
        dtype = sorted_x.dtype
        full_x = np.concatenate(
            ([np.array(0.0, dtype=dtype)], sorted_x, [np.array(1.0, dtype=dtype)])
        )
        full_y = np.concatenate(
            ([np.array(0.0, dtype=dtype)], sorted_y, [np.array(1.0, dtype=dtype)])
        )
        # Remove duplicates based on x-values
        unique_full_x, unique_indices_full = np.unique(full_x, return_index=True)
        if len(unique_full_x) < len(full_x):
            #print('Note: Removing duplicate x-values after adding boundaries.')
            full_x = full_x[unique_indices_full]
            full_y = full_y[unique_indices_full]
    else:
        # Only boundary points if input is empty
        full_x = np.array([0.0, 1.0])
        full_y = np.array([0.0, 1.0])

    # Handle case where only one unique point remains (e.g., input was just [0])
    if len(full_x) < 2:
        # Area for (0,0) to (1,1) with capping:
        # xi=0, yi=0, x_next=1, y_next=1. yi <= y_next.
        # y_calc = 0 + 3*(1-0) = 3. y_calc > y_next. Case 2b.
        # x_hit = 0 + (1-0)/3 = 1/3. w1 = 1/3. w2 = 1 - 1/3 = 2/3.
        # Area = (0+1)/2 * w1 + 1 * w2 = 0.5 * (1/3) + 1 * (2/3) = 1/6 + 4/6 = 5/6
        area = 5.0 / 6.0
        max_gap = 1.0
        # Check if the single gap [0, 1) falls in the range
        if gap_range_min <= 0.0 < gap_range_max:
             max_gap_in_range = 1.0
        else:
             # No gap starts within the range if the only gap starts at 0 and 0 is excluded
             max_gap_in_range = 0.0 # Or return a specific indicator like None or np.nan? Sticking with 0.0 for now.

        # Return 1.0 if no gap qualified, consistent with previous versions' default
        # Let's return the actual max gap (1.0) if it qualifies, else 0.0.
        # If we want to return 1.0 default, uncomment the line below
        # max_gap_in_range = max_gap_in_range if max_gap_in_range > 0 else 1.0
        return area, max_gap_in_range


    # --- Calculate Area segment by segment ---
    total_area = 0.0
    slope = 3.0
    epsilon = 1e-9 # For floating point comparisons

    for i in range(len(full_x) - 1):
        xi = full_x[i]
        yi = full_y[i]
        x_next = full_x[i+1]
        y_next = full_y[i+1]

        w = x_next - xi
        if w < epsilon: # Skip zero-width segments
            continue

        # Case 1: yi > y_next (Use user's rule: horizontal at yi)
        # Use epsilon for robust comparison
        if yi > y_next + epsilon :
            segment_area = yi * w
        # Case 2: yi <= y_next (within epsilon tolerance)
        else:
            # Calculate potential y value if slope 3 continues uninterrupted
            y_calc = yi + slope * w

            # Subcase 2a: Slope 3 does NOT overshoot y_next (y_calc <= y_next)
            if y_calc <= y_next + epsilon:
                # Area is trapezoid under slope 3 line: heights yi and y_calc
                segment_area = (yi + y_calc) * w / 2.0
                # Alternative: segment_area = yi * w + 0.5 * slope * w**2

            # Subcase 2b: Slope 3 overshoots y_next (y_calc > y_next)
            else:
                # Calculate width of the initial slope-3 part until y hits y_next
                # w1 = x_hit - xi = (y_next - yi) / slope
                delta_y = y_next - yi
                if delta_y < 0: delta_y = 0 # Should not happen if yi<=y_next, but safe guard

                # Avoid division by zero if slope is ever zero
                if abs(slope) < epsilon:
                     if delta_y > epsilon: # Cannot reach y_next if slope is zero and yi<y_next
                         warnings.warn(f"Slope is near zero ({slope}), cannot reach y_next={y_next} from yi={yi}")
                         # Treat as if y_calc = yi <= y_next (Subcase 2a with slope=0)
                         segment_area = (yi + yi) * w / 2.0 # Area = yi * w
                     else: # yi is already at (or above) y_next, delta_y approx 0
                         w1 = 0.0 # No slope-3 part needed
                else:
                    w1 = delta_y / slope

                # Clamp w1 to be within [0, w] due to potential floating point issues
                w1 = max(0.0, min(w1, w))
                w2 = w - w1 # Width of the horizontal part at y_next

                # Area = Area of trapezoid (slope 3 part) + Area of rectangle (horizontal part)
                # Trapezoid: (xi, yi) -> (xi+w1, y_next)
                area1 = (yi + y_next) * w1 / 2.0
                # Rectangle: (xi+w1, y_next) -> (x_next, y_next)
                area2 = y_next * w2
                segment_area = area1 + area2

        total_area += segment_area

    # --- Calculate Max Gap (same logic as before) ---
    gaps = np.diff(full_x)
    # Find indices i where the gap starts: gap_range_min <= full_x[i] < gap_range_max
    indices_in_range = np.where(
        (full_x[:-1] >= gap_range_min) & (full_x[:-1] < gap_range_max)
    )[0]

    if indices_in_range.size > 0:
        relevant_gaps = gaps[indices_in_range]
        max_gap_in_range = np.max(relevant_gaps)
    else:
        # No gap starts within the specified range
        # Return 0.0 to indicate no qualifying gap found
        max_gap_in_range = 0.0
        # If a default value like 1.0 is desired when no gap qualifies:
        # max_gap_in_range = 1.0

    return total_area, max_gap_in_range


def get_score(solutions, max_length):
  """Returns the score for the given solutions."""
  solutions = np.array(solutions)

  for i, solution in enumerate(solutions):
    if len(solution) != max_length:
      return -10_000_001
    if not all(isinstance(x, float) for x in solution):
      return -10_000_002
    if not all(0 <= x <= 1 for x in solution):
      return -10_000_003
    if np.sum(solution) < 1e-7:
      return -10_000_004
    solutions[i] = solution / np.sum(solution)

  edge_densities, triangle_densities = sum_pairwise_triple_products_batch(
      solutions
  )
  if np.any(np.isnan(edge_densities)) or np.any(np.isnan(triangle_densities)):
    return -10_000_005

  area, max_gap_in_range = analyze_density_curve(
      edge_densities, triangle_densities
  )
  # print(f"Solution {i + 1}: Area = {area}, Max Gap = {max_gap_in_range}")
  # print(f"Edge Densities: {edge_densities}")
  # print(f"Triangle Densities: {triangle_densities}")
  return -(area + 10 * max_gap_in_range)

def plot_density_curves(
        best_solutions: np.ndarray,
    num_g3_points: int = 500
    ):
    """
    Plots simulation data, theoretical g_3(rho), and the piecewise slope-3
    function f(x) capped by y_{i+1}. Uses NaN separation for discontinuities.
    """
    print("Calculating densities...")
    if best_solutions is None:
         # If no solutions provided, create empty arrays
         edge_densities, triangle_densities = np.array([]), np.array([])
         print("No simulation data provided. Plotting theoretical curves only.")
    else:
        # Calculate densities from solutions
        edge_densities, triangle_densities = sum_pairwise_triple_products_batch(best_solutions)
        print(f"Calculated {len(edge_densities)} density pairs.")

    # --- Prepare Simulation Data for Plotting (Points only) ---
    if edge_densities.size > 0:
        sort_indices = np.argsort(edge_densities)
        sorted_x = edge_densities[sort_indices]
        sorted_y = triangle_densities[sort_indices]
        dtype = sorted_x.dtype
        # Add boundaries (0,0) and (1,1) to the list of points defining f
        f_defining_x = np.concatenate(
            ([np.array(0.0, dtype=dtype)], sorted_x, [np.array(1.0, dtype=dtype)])
        )
        f_defining_y = np.concatenate(
            ([np.array(0.0, dtype=dtype)], sorted_y, [np.array(1.0, dtype=dtype)])
        )
        # Remove duplicates based on x-values
        unique_f_x, unique_indices_f = np.unique(f_defining_x, return_index=True)
        if len(unique_f_x) < len(f_defining_x):
            #print("Note: Removing duplicate x-values from simulation data for defining f(x).")
            f_defining_x = f_defining_x[unique_indices_f]
            f_defining_y = f_defining_y[unique_indices_f]
        # Ensure sorting just in case unique disturbed it (it shouldn't usually)
        sort_check_indices = np.argsort(f_defining_x)
        f_defining_x = f_defining_x[sort_check_indices]
        f_defining_y = f_defining_y[sort_check_indices]
    else:
        # Only boundary points if input is empty
        f_defining_x = np.array([0.0, 1.0])
        f_defining_y = np.array([0.0, 1.0])


    # --- Prepare Capped Slope-3 f(x) Plot Data ---
    f_plot_x_list = []
    f_plot_y_list = []
    slope = 3.0
    epsilon = 1e-9 # For floating point comparisons

    if len(f_defining_x) >= 2:
        # Add the very first point
        f_plot_x_list.append(f_defining_x[0])
        f_plot_y_list.append(f_defining_y[0])

        for i in range(len(f_defining_x) - 1):
            xi = f_defining_x[i]
            yi = f_defining_y[i]
            x_next = f_defining_x[i+1]
            y_next = f_defining_y[i+1]

            w = x_next - xi
            if w < epsilon: # Skip zero-width segments in plotting
                continue

            current_segment_x = []
            current_segment_y = []

            # Case 1: yi > y_next
            if yi > y_next + epsilon:
                # Horizontal line from (xi, yi) to (x_next, yi)
                current_segment_x.extend([xi, x_next])
                current_segment_y.extend([yi, yi])
            # Case 2: yi <= y_next
            else:
                y_calc = yi + slope * w
                # Subcase 2a: y_calc <= y_next (Slope 3 doesn't overshoot)
                if y_calc <= y_next + epsilon:
                    # Slope 3 line from (xi, yi) to (x_next, y_calc)
                    current_segment_x.extend([xi, x_next])
                    current_segment_y.extend([yi, y_calc])
                # Subcase 2b: y_calc > y_next (Slope 3 overshoots)
                else:
                    # Calculate x_hit
                    delta_y = y_next - yi
                    if delta_y < 0: delta_y = 0 # Safety

                    if abs(slope) < epsilon:
                         # Should not happen here based on logic, treat as horizontal yi
                         w1 = 0.0
                    else:
                        w1 = delta_y / slope

                    # Clamp w1 and calculate x_hit
                    w1 = max(0.0, min(w1, w))
                    x_hit = xi + w1

                    # Plot points: (xi, yi) -> (x_hit, y_next) -> (x_next, y_next)
                    current_segment_x.extend([xi, x_hit])
                    current_segment_y.extend([yi, y_next])
                    # Add the endpoint only if x_hit is not essentially x_next
                    if x_next - x_hit > epsilon:
                        current_segment_x.append(x_next)
                        current_segment_y.append(y_next)

            # Add the calculated points for this segment
            # We need to be careful about duplicate x-coordinates at segment boundaries
            # The first point (xi, yi) should already be there from the previous segment's end (unless it was NaN)
            # or from the initial point added.

            # Check if the start of the current segment plot matches the end of the previous plot point
            add_start_point = True
            if f_plot_x_list: # If list is not empty
                 # If last x is same as current xi AND last y was not NaN
                 if np.isclose(f_plot_x_list[-1], xi) and not np.isnan(f_plot_y_list[-1]):
                       # If last y is also same as current yi, we don't need to add start point
                       if np.isclose(f_plot_y_list[-1], yi):
                              add_start_point = False
                       # Else (y value jumped), we still need to add (xi, yi)
                       # In this case, the previous point was likely the end of a plotted segment
                       # before the jump to the required yi. Keep add_start_point = True.
                 # Else (last x different or last y was NaN), we need to add the start point
                 # Keep add_start_point = True

            # Add segment points (excluding the first point if it's redundant)
            if add_start_point:
                 f_plot_x_list.extend(current_segment_x)
                 f_plot_y_list.extend(current_segment_y)
            else:
                 f_plot_x_list.extend(current_segment_x[1:]) # Skip first point
                 f_plot_y_list.extend(current_segment_y[1:]) # Skip first point


            # Add NaN separator before the *next* segment starts, unless the function is continuous
            # Continuity happens ONLY in Case 2b where segment ends at (x_next, y_next)
            is_continuous_at_end = False
            if not (yi > y_next + epsilon) and (yi + slope * w > y_next + epsilon): # Case 2b occurred
                 is_continuous_at_end = True

            # If not continuous, add NaN break point at x_next
            if not is_continuous_at_end:
                 # Check if last plotted point is already at x_next
                 if not np.isclose(f_plot_x_list[-1], x_next):
                      # This shouldn't happen if logic above is correct, but as safety:
                      f_plot_x_list.append(x_next)
                      # What y to use? The end of the *plotted* segment
                      f_plot_y_list.append(f_plot_y_list[-1]) # Take last y before adding NaN

                 f_plot_x_list.append(x_next)
                 f_plot_y_list.append(np.nan)


    else: # Handle case with only (0,0) and (1,1) defined
         xi, yi = 0.0, 0.0
         x_next, y_next = 1.0, 1.0
         # Rule: yi <= y_next. y_calc=3. y_calc > y_next. Case 2b.
         w1 = (y_next - yi) / slope # = (1-0)/3 = 1/3.
         x_hit = xi + w1 # = 1/3.
         f_plot_x_list.extend([xi, x_hit, x_next])
         f_plot_y_list.extend([yi, y_next, y_next]) # (0,0) -> (1/3, 1) -> (1, 1)

    f_plot_x_np = np.array(f_plot_x_list)
    f_plot_y_np = np.array(f_plot_y_list)

    # --- Prepare g_3(rho) Data ---
    g3_rho = np.linspace(0.0, 1.0, num_g3_points)
    g3_y = calculate_g3_vectorized(g3_rho)

    # --- Plotting ---
    print("Generating plot...")
    plt.figure(figsize=(10, 10))

    # Plot original simulation points (the defining points for f)
    # Plot these as scatter points, not connected lines
    plt.scatter(f_defining_x, f_defining_y, marker='o', s=3, facecolors='none', edgecolors='blue',
                label='Data Points (xi, yi)', zorder=4)
    # Optional: Plot linear interpolation of data points for comparison
    # plt.plot(f_defining_x, f_defining_y, linestyle=':', color='blue', alpha=0.5, label='Linear Interpolation')


    # Plot the new capped f(x) using NaN masking
    masked_f_y = np.ma.masked_invalid(f_plot_y_np)
    plt.plot(f_plot_x_np, masked_f_y, linestyle='-', marker='.', markersize=4, color='green',
            label='f(x) (Capped Slope 3 Segments)', zorder=3)

    # Plot g_3(rho)
    plt.plot(g3_rho, g3_y, marker='', linestyle='-', color='red', linewidth=1.5,
            label='g_3(ρ) (Theoretical Bound)', zorder=2)

    # NEW: INCREASED FONT SIZE FOR AXIS TICK LABELS
    plt.tick_params(axis='both', which='major', labelsize=14)

    # Plot settings
    # INCREASED FONT SIZE HERE
    plt.xlabel("Edge Density (ρ)", fontsize=18)
    plt.ylabel("Triangle Density", fontsize=18)
    plt.title("Triangle Density vs. Edge Density (Capped Slope 3 Function)", fontsize=20)
    # INCREASED FONT SIZE HERE
    plt.legend(loc='upper left', fontsize=18)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xlim(-0.02, 1.02)
    # Determine appropriate ylim dynamically
    min_y = 0
    # Combine all relevant y-values to find max, ignoring NaNs
    all_y_values = np.concatenate([f_defining_y, f_plot_y_np[~np.isnan(f_plot_y_np)], g3_y])
    max_y = np.max(all_y_values) if all_y_values.size > 0 else 1.0
    # Add some padding to ylim
    y_padding = max(0.02, (max(1.0, max_y) - min_y) * 0.05)
    plt.ylim(min_y - y_padding, max(1.0, max_y) + y_padding)

    plt.show()
    print("Plot displayed.")






def calculate_g3_vectorized(rho: np.ndarray) -> np.ndarray:
    """
    Calculates the upper bound g_3(rho) using the provided formula,
    determining the correct 't' for each rho.
    Handles rho < 0.5 separately (bound is often 0 there, or not applicable).
    """
    g3_vals = np.zeros_like(rho, dtype=float)

    # Mask for rho values where the formula is potentially applicable (rho >= 0.5)
    # Add small epsilon to avoid issues exactly at 1.0 if present
    valid_rho_mask = (rho >= 0.5) & (rho < 1.0)
    rho_valid = rho[valid_rho_mask]

    if rho_valid.size > 0:
        # Determine the integer t for each rho in the valid range
        # t = floor(1 / (1 - rho))
        # Add epsilon for numerical stability near boundaries
        t = np.floor(1.0 / (1.0 - rho_valid + 1e-15))
        t = np.maximum(2, t).astype(int) # Ensure t >= r-1 = 2

        # Pre-calculate terms for clarity and potential efficiency
        t_float = t.astype(float) # Use float t for calculations
        t_plus_1 = t_float + 1.0

        # Term inside sqrt: t*(t - rho*(t+1))
        sqrt_term_inner = t_float * (t_float - rho_valid * t_plus_1)
        # Handle potential small negative values due to precision
        sqrt_term_inner = np.maximum(0.0, sqrt_term_inner)
        sqrt_term = np.sqrt(sqrt_term_inner)

        # Term 1: (t-1) / (t**2 * (t+1)**2)
        term1_num = t_float - 1.0
        term1_den = (t_float**2) * (t_plus_1**2)
        # Avoid division by zero if t=0 somehow occurred (shouldn't with t>=2)
        term1 = np.divide(term1_num, term1_den, out=np.zeros_like(term1_num), where=term1_den!=0)

        # Term 2: [t - 2*sqrt(term)]
        term2 = t_float - 2.0 * sqrt_term

        # Term 3: [t + sqrt(term)]^2
        term3 = np.square(t_float + sqrt_term)

        # Combine terms
        g3_valid = term1 * term2 * term3

        # Ensure result is non-negative (it should be theoretically)
        g3_valid = np.maximum(0.0, g3_valid)

        # Place calculated values back into the full array
        g3_vals[valid_rho_mask] = g3_valid

    # Special handling for rho >= 1 if needed, often g_r(1)=1
    g3_vals[rho >= 1.0] = 1.0 # Assuming this limit based on context

    return g3_vals

In [ ]:
#@title Data, verification, visualization

best_solutions = np.array([[1.616699982783082e-01, 1.543057328542216e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.840242688674702e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.219183693958312e-01, 1.603741196724814e-01, 1.501643966383713e-01, 6.754311429331619e-02], [3.221469073106969e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.351852954060235e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.295911808464399e-01, 0.000000000000000e+00,0.000000000000000e+00,1.307661643683977e-02], [4.266801412961053e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.216023483569766e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.517175103469180e-01], [2.053195825093272e-01, 1.845528962862067e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.941606352151726e-01, 1.942880609465776e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.567656367662728e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.060022613660887e-01, 0.000000000000000e+00], [1.718088080506412e-01, 1.467120074624758e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.543497671173445e-01, 1.544510653642592e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.222890203633533e-01, 0.000000000000000e+00,1.293448104349620e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.210445212069640e-01, 0.000000000000000e+00], [1.480797165346565e-01, 1.413345050043918e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.314274243449110e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.519168694251984e-01, 1.211924649160464e-01, 1.115407700174985e-01, 9.618347066297028e-02, 0.000000000000000e+00,9.832477909432724e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.307935302040479e-01, 0.000000000000000e+00,1.334582443519229e-01, 0.000000000000000e+00,1.077491511726836e-01, 1.026867665553470e-01, 6.738870824208981e-02, 9.218620123354912e-02, 0.000000000000000e+00,5.360433146327811e-02, 9.616701339757661e-02, 6.514538610220780e-04, 2.842658943585438e-03, 0.000000000000000e+00,9.039913428728157e-02, 1.220728062876160e-01], [7.589377580308538e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.410622419691462e-01], [6.910483485793051e-02, 4.974070105974455e-02, 4.950520487090557e-02, 4.935929395351854e-02, 4.956311075289233e-02, 7.202522241865301e-02, 9.586143695169781e-02, 5.779957549565580e-02, 5.158551452351586e-02, 0.000000000000000e+00,7.358958794719671e-02, 1.006688618933506e-01, 0.000000000000000e+00,5.853681970570529e-02, 1.050159748517053e-01, 7.113984293481744e-04, 3.104230750074080e-03, 0.000000000000000e+00,6.419239335808694e-02, 4.963583818001899e-02], [2.457535004247804e-01, 2.345591290182132e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.472979470453056e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.538096476935505e-01, 1.238114863747690e-02, 6.198627180673493e-03], [3.233399965566163e-01, 3.086114657084432e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.680485377349404e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.684880370257584e-03, 7.190479186962100e-02, 7.900873530038552e-02, 8.297383110107345e-02, 8.195528029530881e-02, 8.858223147688750e-02, 8.409628032582404e-02, 8.097505297804269e-02, 8.467898093712070e-02, 9.013451753026902e-02, 8.356594517568533e-02, 8.334569229869811e-02, 8.278084149299907e-02, 2.312938847827223e-03, 0.000000000000000e+00,0.000000000000000e+00], [4.171863917444897e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.633839426768047e-01, 0.000000000000000e+00,3.194296655787057e-01], [4.897552822254472e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.879849129419983e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.225980483255457e-02, 0.000000000000000e+00,0.000000000000000e+00], [2.293662684801766e-01, 2.189183554573003e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.308077289582751e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.204096231478373e-01, 2.004980239564107e-01, 0.000000000000000e+00,0.000000000000000e+00], [3.989023553575372e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.216370184882347e-01, 0.000000000000000e+00,0.000000000000000e+00,1.794606261542281e-01], [2.344739181045496e-01, 2.237933454173714e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.359474777885090e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.429763783231698e-01, 0.000000000000000e+00,6.280888036640025e-02, 0.000000000000000e+00], [2.482678040126902e-01, 2.369589030138964e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.251766140361838e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.495886203789349e-01, 0.000000000000000e+00,0.000000000000000e+00,4.000805855829479e-02], [2.108942219134782e-01, 2.043116792555483e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.899901072799307e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.196094343595973e-01, 1.751945571914455e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [3.336268781675120e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.359903361069463e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.303827857255417e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [4.837819837332583e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.762676204798203e-01, 0.000000000000000e+00,0.000000000000000e+00,3.995039578692149e-02], [2.732799648167456e-01, 2.806279749636970e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.749359573056871e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.711561029138704e-01, 0.000000000000000e+00], [3.008321124813432e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.130078178662338e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.077838369363188e-01, 0.000000000000000e+00,0.000000000000000e+00,7.837623271610407e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,5.588041373183431e-02, 9.441195862681657e-01, 0.000000000000000e+00], [2.456164644415056e-01, 2.344283351909614e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.471600498545163e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.312041151793315e-02, 0.000000000000000e+00,1.796747389950836e-01, 0.000000000000000e+00], [9.777655325420783e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.223446745792172e-02], [4.668331966882094e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.635497414593863e-01, 0.000000000000000e+00,0.000000000000000e+00,6.961706185240428e-02], [1.154569744772111e-01, 1.037792828905834e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.091819846433738e-01, 1.092536397151344e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.597703900047773e-02, 7.478618280191714e-02, 4.567446376735678e-02, 6.250552687630881e-02, 0.000000000000000e+00,1.150536079036460e-01, 7.851656546355659e-02, 8.981473246043439e-02, 0.000000000000000e+00], [1.533482474405494e-01, 1.309480548647081e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.377651503951132e-01, 1.378555643198090e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.397500989713012e-02, 0.000000000000000e+00,1.100103110672923e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.450311413057912e-01, 1.663020259984476e-01, 1.037737746717222e-01], [2.458184792370227e-01, 2.346211479663781e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.473633342192062e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.721970385773930e-01, 0.000000000000000e+00,0.000000000000000e+00], [1.338194119923476e-01, 1.277237747108380e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.560032871448503e-01, 1.374880320410287e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.583547535733077e-01, 1.144228031272007e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.091328096791357e-01, 6.305512773129131e-02], [1.007029974872034e-01, 1.022376930126454e-01, 2.206641983786378e-02, 7.344876380679712e-02, 0.000000000000000e+00,2.739465965591325e-02, 8.935755043248939e-02, 6.644698665248878e-04, 3.715247508377899e-02, 0.000000000000000e+00,8.157806881467004e-02, 1.124163191192990e-01, 0.000000000000000e+00,0.000000000000000e+00,4.425320395833497e-02, 1.267605174716687e-01, 0.000000000000000e+00,0.000000000000000e+00,1.171441677558138e-01, 6.482269369699734e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,7.877428459245156e-01, 0.000000000000000e+00,2.122571540754844e-01, 0.000000000000000e+00], [2.890106151555579e-01, 2.758458294621715e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.908269126532593e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.443166427290113e-01, 0.000000000000000e+00], [3.678621241219278e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,6.321378758780721e-01, 0.000000000000000e+00,0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.858875160088100e-01, 0.000000000000000e+00,4.090014635380119e-01, 0.000000000000000e+00,2.051110204531780e-01], [1.783790323342217e-01, 1.532245387055562e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.612013377426799e-01, 1.613071326085783e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.301542422891263e-02, 0.000000000000000e+00,1.024565190747729e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.343530992856723e-01, 9.011768779511381e-03, 1.792620144073903e-01, 0.000000000000000e+00], [4.863689911712528e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.862888250125630e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.734218381618430e-02, 0.000000000000000e+00], [4.400884405839033e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.400159026540480e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.198956567620488e-01], [7.096249501493478e-02, 5.107781905923201e-02, 5.083599231641613e-02, 5.068615905555981e-02, 5.089545481090395e-02, 7.396138775732392e-02, 9.843836188588581e-02, 5.935333029024840e-02, 5.297222437103824e-02, 0.000000000000000e+00,0.000000000000000e+00,7.230213129827154e-02, 2.922976155074553e-02, 0.000000000000000e+00,5.966447180162376e-02, 7.674014267395218e-02, 8.639178059423415e-02, 8.274681702675350e-02, 3.374167049287634e-02, 0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.288056618463641e-01, 3.779128482117600e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.932814899418759e-01, 0.000000000000000e+00,0.000000000000000e+00], [4.996794793098901e-02, 5.006107315247825e-02, 4.998559611653128e-02, 4.995080688164900e-02, 5.000232344969021e-02, 5.001359391413348e-02, 4.996512587007753e-02, 5.000080997672408e-02, 4.999761047483434e-02, 5.007995390213629e-02, 4.988896735425338e-02, 5.002780947603458e-02, 5.001718452605530e-02, 5.001854811606320e-02, 4.999048267048197e-02, 5.000978445807944e-02, 5.002139418198893e-02, 4.995709350779211e-02, 4.994915182233473e-02, 5.009474221767291e-02], [3.180497946975858e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.205052206479130e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.381903669181699e-01, 0.000000000000000e+00,0.000000000000000e+00,2.325461773633131e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.749175184303061e-01, 1.986584432619842e-01, 1.794138872404755e-01, 1.799679335842623e-01, 0.000000000000000e+00,0.000000000000000e+00,9.606470016974732e-02, 1.709775173132245e-01], [1.119676004258696e-01, 9.199975458653258e-02, 1.163604474409536e-01, 0.000000000000000e+00,9.201666252171538e-02, 3.132202524579530e-02, 4.070394282549022e-02, 0.000000000000000e+00,3.309112565766113e-02, 7.188299802588975e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.792075735244447e-03, 0.000000000000000e+00,1.051955747503673e-01, 7.507228618386126e-02, 6.521621337341629e-02, 9.737499257874743e-02, 6.600430064845574e-02], [4.443540125643341e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.601221144668068e-01, 0.000000000000000e+00,0.000000000000000e+00,9.552387296885910e-02], [4.398332869741099e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.298638286943248e-01, 0.000000000000000e+00,0.000000000000000e+00,1.303028843315653e-01], [3.277905046013257e-01, 3.366042058014851e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.267364366054823e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,8.868852991706828e-03, 0.000000000000000e+00,0.000000000000000e+00], [3.965596055463514e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.262094812447538e-01, 3.772309132088948e-01, 0.000000000000000e+00,0.000000000000000e+00], [4.293818676731940e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.087661645929135e-01, 0.000000000000000e+00,1.618519677338925e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.254227158789841e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,7.457728412101597e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [2.539019968546615e-01, 2.423364515063865e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.554976529910298e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.403877877810967e-01, 0.000000000000000e+00,0.000000000000000e+00,7.876110866825487e-03], [1.090601948338732e-01, 1.040923700642047e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.271396177687207e-01, 1.120500481841740e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.290560167650602e-01, 9.325233922866161e-02, 0.000000000000000e+00,7.768115925127772e-02, 0.000000000000000e+00,5.155386045788234e-02, 2.648153717148393e-02, 7.922286261498744e-02, 9.040999365967411e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.735459690875908e-01, 8.264540309124092e-01, 0.000000000000000e+00], [4.901571104499240e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.202906855563425e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.978138209944417e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,6.820038020595900e-01, 3.179961979404100e-01, 0.000000000000000e+00], [0.000000000000000e+00,8.231850049644431e-02, 1.181911821172390e-03, 3.769184747124002e-02, 5.606067420024995e-02, 5.432808520882509e-02, 5.968012191425340e-02, 7.072184229261103e-02, 6.197536540345174e-02, 4.106697440365900e-02, 7.981684430597571e-02, 6.345483631714988e-02, 9.446996825204705e-02, 1.642313761457333e-03, 3.906596411952720e-02, 4.096878518601892e-02, 4.712392313732853e-02, 4.203861742758905e-02, 6.804721007617394e-02, 5.834621420482554e-02], [1.274940155168384e-01, 1.216865077525819e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.194065951027556e-01, 1.194849604866486e-01, 0.000000000000000e+00,1.111951746993120e-01, 0.000000000000000e+00,9.017751221104410e-02, 0.000000000000000e+00,7.023017227542040e-02, 0.000000000000000e+00,0.000000000000000e+00,1.403601051441368e-01, 0.000000000000000e+00,0.000000000000000e+00,9.996495681126229e-02, 0.000000000000000e+00], [2.258533603529428e-01, 2.155654645759061e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.272727438348807e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.624879186044310e-01, 0.000000000000000e+00,1.688205126318394e-01, 0.000000000000000e+00,0.000000000000000e+00], [2.102281634986257e-01, 1.889650075283933e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.988024389400756e-01, 1.894141972832080e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.747543843352771e-02, 0.000000000000000e+00,1.375654423332105e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.813582100828486e-01, 0.000000000000000e+00,0.000000000000000e+00], [4.629886401715345e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.566650477087713e-01, 0.000000000000000e+00,7.886084019478028e-02, 1.485471924913691e-03], [3.090339505766482e-01, 3.173433215402768e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.080401975778294e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,6.558253030524547e-02], [3.171942075008773e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.490019739574258e-01, 0.000000000000000e+00,2.633688046871145e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.704350138545823e-01], [3.202169134032709e-01, 3.189084448041599e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.239170612631116e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.626548096166720e-03, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.433103243329091e-02], [4.593297486393575e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.534772921611107e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,8.719295919953181e-02], [1.967609920983341e-01, 1.877982891457161e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.979975435564910e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.831535816328902e-01, 6.188972726977526e-02, 1.584175288507823e-03, 1.708156910082854e-01], [2.477743108917333e-01, 2.440818712400762e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.268977102692738e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.633669802541270e-01, 0.000000000000000e+00,0.000000000000000e+00,1.178791273447897e-01], [3.178858410306973e-01, 3.264332235182210e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.259883982833098e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.982090617993399e-03, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.671044654977867e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,8.615149304724637e-01, 1.384850695275363e-01, 0.000000000000000e+00], [3.811252414609911e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.592735453261077e-01, 2.596012132129012e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.015265586183390e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.847344138166109e-02, 0.000000000000000e+00,0.000000000000000e+00], [2.478031050496696e-01, 2.365153716548989e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.286790964418568e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.569517248010110e-01, 0.000000000000000e+00,3.005070205256384e-02], [1.700817185443257e-01, 1.527397956128646e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.865581427372076e-01, 1.644164836241702e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.893701681607141e-01, 1.368336913207177e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [4.856037245101290e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.805697662569086e-01, 0.000000000000000e+00,0.000000000000000e+00,3.382650923296240e-02], [4.891928049671583e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,5.017711320150706e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,9.036063017771080e-03, 0.000000000000000e+00], [3.192555945537156e-01, 3.182358995637005e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.178022446885781e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.907204971875758e-03, 0.000000000000000e+00,0.000000000000000e+00,4.179905622213005e-02, 0.000000000000000e+00], [4.774494162354850e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.773707202528812e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.317817749319280e-02, 1.200168601844099e-02], [2.472742986464810e-01, 2.360106530277511e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.348231992350713e-01, 2.349773114106946e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.895966621389722e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.617026036099474e-03, 2.333784543000524e-02], [2.950531361290210e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.972068374526513e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.136064030579950e-01, 0.000000000000000e+00,9.413362336033282e-02, 0.000000000000000e+00], [4.613796256202699e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.638141179188116e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,7.480625646091851e-02], [4.905843322268349e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.928398216555963e-01, 0.000000000000000e+00,1.657584611756886e-02, 0.000000000000000e+00], [4.427844350909252e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.530103620677742e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.042052028413006e-01, 0.000000000000000e+00], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,7.641588809039460e-02, 7.608643688443481e-02, 7.673866420285846e-02, 7.616882940536532e-02, 7.660472730159493e-02, 7.586291331000192e-02, 7.627799616639805e-02, 7.610957774588682e-02, 7.696542161245611e-02, 7.630514656550555e-02, 7.663073913700209e-02, 7.665403848915041e-02, 8.317962108895101e-02, 0.000000000000000e+00,0.000000000000000e+00], [1.501074655212832e-01, 1.281806669378709e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.348536935251639e-01, 1.349421966891090e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.372621726667977e-01, 3.597546317473341e-02, 0.000000000000000e+00,6.867007572641241e-02, 0.000000000000000e+00,1.273809397606583e-01, 8.262732599797112e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,8.744076310849419e-01, 0.000000000000000e+00,1.255923689150581e-01], [4.066984160211644e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.231588931063109e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.701426908725247e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [4.725516482971623e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.704503247391861e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,5.699802696365158e-02, 0.000000000000000e+00,0.000000000000000e+00], [1.130078776906513e-01, 1.078602311564190e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.304271213380153e-01, 1.305127193858026e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.033354646081990e-01, 0.000000000000000e+00,1.092976789023458e-01, 0.000000000000000e+00,1.094974495246140e-01, 9.761805307074591e-02, 6.975409957880140e-02, 2.868930474440559e-02], [3.170310669769288e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.160512224066144e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.107764482173190e-01, 0.000000000000000e+00,5.614126239913791e-02, 0.000000000000000e+00], [4.432029609447692e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.445428429477704e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.122541961074603e-01, 0.000000000000000e+00], [9.670626151903100e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,3.293738480969004e-02, 0.000000000000000e+00], [1.578936052874791e-01, 1.661210550589605e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.747692408080245e-01, 1.748839401563882e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.571495973985966e-02, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.828419921109487e-01, 0.000000000000000e+00,1.277752068383394e-01], [4.955948648402598e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,5.044051351597403e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [3.015318352904618e-01, 3.002997175552783e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.854518610864963e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,1.127165860677638e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00], [2.840822498080262e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.955800309132728e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.906469130912301e-01, 0.000000000000000e+00,1.296908061874708e-01, 0.000000000000000e+00], [4.877808142032851e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.614526299504636e-01, 0.000000000000000e+00,1.924200929236310e-02, 3.152454655388820e-02], [6.931190374343914e-02, 5.769524717632195e-02, 5.264595968848474e-02, 4.210521070632976e-02, 5.274138577627915e-02, 5.125320134886215e-02, 4.829250631668593e-02, 4.680084611478574e-02, 5.666664889796764e-02, 5.432479854269069e-02, 4.979650704927225e-02, 4.832239906574707e-02, 4.446313883499555e-02, 5.728081496584640e-02, 6.337661782351206e-02, 6.189761615006221e-02, 4.824365647586837e-02, 8.359523653381509e-02, 2.043424711695171e-03, 9.142880077339023e-03], [4.339871927189820e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.241502446299947e-01, 0.000000000000000e+00,1.418625626510232e-01, 0.000000000000000e+00], [4.668061820338874e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,4.692693078243800e-01, 0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,6.392451014173264e-02], [0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,0.000000000000000e+00,2.851315924802448e-01, 0.000000000000000e+00,7.148684075197551e-01]])
print(f"Score: {get_score(best_solutions, 20)}")
plot_density_curves(best_solutions)


In [ ]:
#@title Code evolved by AlphaEvolve

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
import numpy as np

njit = numba.njit

def create_nn_transformer(input_dim, hidden_dim, output_dim):
  """Creates a simple randomly initialized neural network transformer."""
  W1 = np.random.randn(input_dim, hidden_dim) * 0.1  # Small initialization
  b1 = np.zeros(hidden_dim)
  W2 = np.random.randn(hidden_dim, output_dim) * 0.1
  b2 = np.zeros(output_dim)
  return W1, b1, W2, b2

def apply_nn_transformer(solution, W1, b1, W2, b2):
  """Applies the neural network transformation to a solution."""
  hidden_layer = np.maximum(0, solution @ W1 + b1) # ReLU activation
  transformed_solution = hidden_layer @ W2 + b2
  return transformed_solution


def search_for_solutions(max_length):
  # Initialize population
  population_size = 100
  hidden_dim = 64 # Hidden dimension for the NN - can be tuned
  input_dim = max_length
  output_dim = max_length
  W1, b1, W2, b2 = create_nn_transformer(input_dim, hidden_dim, output_dim)


  # Initialize using parametrized families
  solutions_family_1 = create_parametrized_family_3(0.1, 0.5, 0.5, 5, 0, np.pi/2, max_length, population_size//3)
  solutions_family_2 = create_parametrized_family_1(0.1, 0.3, 0.1, 0.3, 0.05, 0.05, max_length, population_size//3)
  solutions_family_3 = create_parametrized_family_2(0.1, 0.3, 0.1, 0.3, 0.05, 0.05, max_length, population_size//3)

  best_solutions_initial = np.concatenate([solutions_family_1, solutions_family_2, solutions_family_3])
  best_solutions = np.zeros_like(best_solutions_initial)
  for i in range(len(best_solutions_initial)):
      transformed_solution = apply_nn_transformer(best_solutions_initial[i], W1, b1, W2, b2)
      best_solutions[i] = np.clip(transformed_solution, 0, 1) # Clip after transformation
      best_solutions[i] /= np.sum(best_solutions[i]) # Normalize again


  remaining = population_size - len(best_solutions)
  if remaining > 0:
    best_solutions = np.concatenate([best_solutions, np.random.rand(remaining, max_length)])

  best_solutions = best_solutions / np.sum(best_solutions, axis=1, keepdims=True)
  best_solutions = best_solutions[:population_size]

  best_score = get_score(best_solutions, max_length)

  start_time = time.time()
  eval_count = 0
  mutation_rate = 0.01

  temperature = 1.0
  cooling_rate = 0.99

  while time.time() - start_time < 197:
    # Create new generation
    new_solutions = best_solutions.copy()
    for i in range(population_size):

      # Mutation

      # "Dreaming" mutation: Project to frequency space, manipulate, project back
      if np.random.rand() < mutation_rate / 4:
          fft_solution = np.fft.fft(new_solutions[i])
          num_components = len(fft_solution)
          component_indices = np.random.choice(num_components, size=num_components // 2, replace=False)
          amplitude_multiplier = np.ones(num_components)
          amplitude_multiplier[component_indices] = np.random.uniform(0.5, 1.5, len(component_indices))
          phase_shift = np.zeros(num_components)
          phase_shift[component_indices] = np.random.uniform(-np.pi/8, np.pi/8, len(component_indices))

          fft_solution = fft_solution * amplitude_multiplier * np.exp(1j*phase_shift)
          mutated_solution = np.fft.ifft(fft_solution).real
          new_solutions[i] = np.clip(mutated_solution, 0, 1)
          new_solutions[i] /= np.sum(new_solutions[i])

      if np.random.rand() < mutation_rate / 2:
          family_choice = np.random.choice([1, 2, 3])
          family = [solutions_family_1, solutions_family_2, solutions_family_3][family_choice - 1]
          if len(family) > 0:
              new_solutions[i] = family[np.random.randint(0, len(family))]
          else:
              new_solutions[i] = np.random.rand(max_length)

      if np.random.rand() < mutation_rate:
          crossover_partner_idx = np.random.randint(0, population_size)
          crossover_point = np.random.randint(1, max_length - 1)
          new_solutions[i][:crossover_point] = best_solutions[np.random.choice(population_size)][:crossover_point]
          new_solutions[i][crossover_point:] = best_solutions[np.random.choice(population_size)][crossover_point:]


      if np.random.rand() < mutation_rate:
          noise = np.random.normal(0, 0.1 * temperature, max_length)
          new_solutions[i] = np.clip(new_solutions[i] + noise, 0, 1)
          new_solutions[i] /= np.sum(new_solutions[i])

       # Heuristic Mutation: Adjust to improve normalization.
      if np.random.rand() < mutation_rate / 4:
        adjustment = np.random.normal(0, 0.01 * temperature, max_length)
        new_solutions[i] += adjustment
        new_solutions[i] = np.clip(new_solutions[i], 0, 1)
        new_solutions[i] /= np.sum(new_solutions[i])


      # Solution Surgery
      if np.random.rand() < mutation_rate / 2:
          partner_idx = np.random.randint(0, population_size)
          length = max_length // 5  # Adjust for region size
          start1 = np.random.randint(0, max_length - length)
          start2 = np.random.randint(0, max_length - length)

          # Simple region similarity (average value)
          region1 = new_solutions[i][start1:start1 + length]
          region2 = best_solutions[partner_idx][start2:start2 + length]

          if np.abs(np.mean(region1) - np.mean(region2)) < 0.1: # if similar enough
              new_solutions[i][start1:start1 + length] = region2

    mutation_rate = max(0.001, 0.01 - (time.time() - start_time) / 20000)

    score = get_score(new_solutions, max_length)
    eval_count += 1

    if score > best_score:
      best_score = score
      best_solutions = new_solutions
    else:
      # Simulated Annealing acceptance criterion
      acceptance_probability = np.exp((score - best_score) / temperature)
      if np.random.rand() < acceptance_probability:
        best_solutions = new_solutions
        best_score = score

    temperature *= cooling_rate

  print(f'Final score: {best_score}')
  print(f'Evaluations: {eval_count}')
  print(f'Best final returned solutions: {best_solutions}')
  return best_solutions


def create_parametrized_family_3(
    amplitude_min,
    amplitude_max,
    frequency_min,
    frequency_max,
    phase_min,
    phase_max,
    max_length,
    num_solutions
):
  """Creates a family of solutions based on sine and cosine waves."""

  solutions = np.zeros((num_solutions, max_length))

  for i in range(num_solutions):
    num_waves = np.random.randint(1, 6)  # Variable number of waves
    amplitude = np.random.uniform(amplitude_min, amplitude_max, num_waves)
    frequency = np.random.uniform(frequency_min, frequency_max, num_waves)
    phase = np.random.uniform(phase_min, phase_max, num_waves)

    x = np.linspace(0, 2 * np.pi, max_length)
    cumulative_sum = np.zeros_like(x)
    for j in range(num_waves):
      cumulative_sum += amplitude[j] * np.sin(frequency[j] * x + phase[j])
      cumulative_sum += amplitude[j] * np.cos(frequency[j] * x + phase[j])
    solutions[i] = cumulative_sum

  solutions = np.clip(solutions, 0, 1)
  solutions = solutions / np.sum(solutions, axis=1, keepdims=True)
  return solutions


def create_parametrized_family_1(
    param_1_min,
    param_1_max,
    param_2_min,
    param_2_max,
    param_1_step,
    param_2_step,
    max_length,
    num_solutions
):
  """Creates a family of solutions based on linear combinations."""

  param1_values = np.random.uniform(param_1_min, param_1_max, num_solutions)
  param2_values = np.random.uniform(param_2_min, param_2_max, num_solutions)

  solutions = np.zeros((num_solutions, max_length))

  for i in range(num_solutions):
    solutions[i] = np.linspace(param1_values[i], param2_values[i], max_length)
  solutions = np.clip(solutions, 0, 1)
  solutions = solutions / np.sum(solutions, axis=1, keepdims=True)
  return solutions


def create_parametrized_family_2(
    param_1_min,
    param_1_max,
    param_2_min,
    param_2_max,
    param_1_step,
    param_2_step,
    max_length,
    num_solutions
):
  """Creates a family of solutions based on exponential decay."""

  param1_values = np.random.uniform(param_1_min, param_1_max, num_solutions)
  param2_values = np.random.uniform(param_2_min, param_2_max, num_solutions)

  solutions = np.zeros((num_solutions, max_length))

  for i in range(num_solutions):
    solutions[i] = param1_values[i] * np.exp(-param2_values[i] * np.linspace(0, max_length, max_length))
  solutions = np.clip(solutions, 0, 1)
  solutions = solutions / np.sum(solutions, axis=1, keepdims=True)
  return solutions

**Prompt used**

You are an expert in optimization. You are given a particularly challenging task: you are given one parameter, 'max_length', and your task is to create a python or numpy array containing many lists of length 'max_length'. Each of these lists will be the solution to a secret optimization problem. All numbers have to be between 0 and 1. You don't know what exactly these optimization problems are, but they are closely related, and changing one list will affect the score of the others.

You are expected to return on the order of about 1000 lists. It will be hard to optimize these lists one by one. Instead you should generate many lists at once using parametrized families, and optimize these functions and the parameters therein. Here is an example of a parametrized family generating function:
```
def create_parametrized_family_1(
    param_1_min,
    param_1_max,
    param_2_min,
    param_2_max,
    param_1_step,
    param_2_step,
    max_length,
):
  """Creates a family of solutions based on two parameters.

Args:
    param_1_min: Minimum value for the first parameter.
    param_1_max: Maximum value for the first parameter.
    param_2_min: Minimum value for the second parameter.
    param_2_max: Maximum value for the second parameter.
    param_1_step: Step size for the first parameter.
    param_2_step: Step size for the second parameter.
    max_length: The desired length of each solution.

Returns:
    A 2D numpy array of shape (N, max_length) where N is the number of
    parameter combinations. Each row is a solution based on the parameter
    values, padded with zeros to reach max_length.
  """

solutions = []
  for param_1 in np.arange(param_1_min, param_1_max, param_1_step):
    for param_2 in np.arange(param_2_min, param_2_max, param_2_step):
      solution = [param_1, param_1, param_2, param_1 + param_2]
      # pad it with zeros until it reaches max_length
      solution += [0] * (max_length - len(solution))
      solution = solution / np.sum(solution)
      solutions.append(solution)
  return np.array(solutions)
```
This parametrized family is not suitable for the problem, some other parametrization will be needed. You will have to guess the right form, but you can expect it to be something reasonably simple. You should use more than just the first 4 entries in each list.

Your final solution will be evaluated by
an evaluation function that looks like this, with the helper functions implemented elsewhere in the code but inaccessible for us:

```
def get_score(solutions, num_problems, max_length):
  """Returns the score for the given solutions."""
  if len(solutions) > 100:
    solutions = solutions[:100]
  solutions = np.array(solutions)
  if len(solutions) != num_problems:
    return -10_000_000

for i, solution in enumerate(solutions):
    if len(solution) != max_length:
      return -10_000_001
    if not all(isinstance(x, float) for x in solution):
      return -10_000_002
    if not all(0 <= x <= 1 for x in solution):
      return -10_000_003
    if np.sum(solution) < 1e-7:
      return -10_000_004
    solutions[i] = solution / np.sum(solution)

edge_densities, triangle_densities = sum_pairwise_triple_products_batch(
      solutions
  )
  if np.any(np.isnan(edge_densities)) or np.any(np.isnan(triangle_densities)):
    return -10_000_005

area, max_gap_in_range = analyze_density_curve(
      edge_densities, triangle_densities
  )
  return -(area + 10 * max_gap_in_range)
```
You may code up any search method you want, and you are allowed to call the
get_score() function as many times as you want. You have
access to it; you don't need to code up the get_score()
function. You want the score it gives you to be as high (as least negative) as
possible!

Your task is to write a search function that searches for the best solution, which should be assembled from several parametrized families. Your function will have 200 seconds to run, and after that, it has
to have returned the best construction it found. If after 200 seconds it has
not returned anything, it will be terminated with negative infinity points.